In [2]:
import pandas as pd
import csv
import os
cur_dir =  os.getcwd() 


In [4]:
#this method decomposes a given csv file and extracts all qualitative answers
def decompose_csv(file_identifier):
    df = pd.read_csv(file_identifier, sep=';', encoding='UTF-16 LE')
    df = df.set_index('CASE')
    df_free_answers = df.filter(regex='AN') 
    df_assessment = df.filter(regex='AS0') 
    df_mike_feedback = df.filter(regex='FB')
    df_qual = pd.concat([df_free_answers, df_mike_feedback, df_assessment], axis=1)
    return df_qual

# this method takes a csv file as input and creates a new txt file for every case
# with the case id, the code of a question, the text of a question and the answers
def create_txt(file_identifier):
    # we drop all the entries without qualitative answers
    df_qual = decompose_csv(file_identifier).dropna()[1:] 
    questions = 'questions.csv'
    df_questions = pd.read_csv(questions, sep=';')
#    codes = list(df_qual.columns.values)
    codes = ['AS01_01','AS01_02', 'AS02_01', 'AS02_02', 'AN01_01', 'AN01_02', 'FB01_01', 'FB01_02', 'FB02_01' ,'FB02_02' ]

    labels = df_questions[:]['Frage'].values.tolist()
   # print(labels)
    likert_questions = ['FB01_01', 'FB02_01', 'AS01_01', 'AS02_01']
  
    for index, row in df_qual.iterrows():
        new_file = 'case_txts/case_{}.txt'.format(index)
        with open(new_file, 'w') as f:
            # write the id of the case
            f.write('case {}:\n'.format(index))
            for i in range(len(row)):
                # get the question corresponding to the questions code
                question = df_questions.loc[df_questions['Code'] == codes[i]]['Frage'].values[0]
                f.write('\n{} - {}\n'.format(codes[i], question))
                # if it is a likert question we have to map the value for answer to the actual answer
                if codes[i] in likert_questions:
                    answers = df_questions.loc[df_questions['Code'] == codes[i]]['Antwortart'].values[0].split(',')
                    this_answer = answers[int(row[codes[i]])-1]
                    f.write('{}\n'.format(this_answer))
                # else it's a freetext answer so we can simply write it down
                else:
                    f.write('{} \n'.format(row[codes[i]]))
        
    return 

In [6]:
parent_dir = os.path.dirname(os.path.dirname(cur_dir))
data_file = 'data/full_experiment/data_furnituremanufacturing_2021-08-11_09-15.csv'
data_path = os.path.join(parent_dir, data_file)

create_txt(data_path)

In [24]:
# we drop all the entries without qualitative answers
df_qual = decompose_csv(data_path).dropna()[1:] 
questions = 'questions.csv'
df_questions = pd.read_csv(questions, sep=';')
#    codes = list(df_qual.columns.values)
codes = ['AS01_01','AS01_02', 'AS02_01', 'AS02_02', 'AN01_01', 'AN01_02', 'FB01_01', 'FB01_02', 'FB02_01' ,'FB02_02' ]

labels = df_questions[:]['Frage'].values.tolist()
# print(labels)
likert_questions = ['FB01_01', 'FB02_01', 'AS01_01', 'AS02_01']
df_qual

for index, row in df_qual.iterrows():
    for i in range(len(row)):
                # get the question corresponding to the questions code
            question = df_questions.loc[df_questions['Code'] == codes[i]]['Frage'].values[0]
            if codes[i] in likert_questions:
                answers = df_questions.loc[df_questions['Code'] == codes[i]]['Antwortart'].values[0].split(',')
                this_answer = answers[int(row[codes[i]])-1]
